In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-10-26 14:57:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.19MB/s    in 0.3s    

2023-10-26 14:57:41 (3.19 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [6]:
#All the unique characters sorted set
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
#Encoding Characters to Integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Hello, Pratyush here"))
print(decode(encode("Hello, Pratyush here")))

[20, 43, 50, 50, 53, 6, 1, 28, 56, 39, 58, 63, 59, 57, 46, 1, 46, 43, 56, 43]
Hello, Pratyush here


In [9]:
#Encoding Entire Dataset stored in Torch Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [10]:
#Splitting for Testing and Training
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
#Creating Chunks of Training Data
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1] #Including target
    target = y[t] #Including upto target

In [13]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

In [16]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # Random tensor ix of size batch_size containing random integers which will be used as starting indices for selecting subsequences from the data.
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Creates tensor x by stacking together subsequences of data
    x = torch.stack([data[i:i+block_size] for i in ix])
    # Creates tensor y by stacking together subsequences of data, but with a one-step offset than x
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1] # extracts a subsequence of the input data xb[b] up to time step t
        target = yb[b,t] # extracts the target value at time step t from the target data yb[b]

In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Create an embedding table for token embeddings.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx tensor representing sequences of token indices
        # targets tensor which is used for training
        # logits has dimensions (B, T, C), where B is the batch size, T is the sequence length, and C is the number of tokens in the vocabulary
        logits = self.token_embedding_table(idx)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            # Calculate the cross-entropy loss between logits and targets.
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # Generate new tokens based on the current context (idx).
        # idx is (B, T) array of indices in the current context.
        for _ in range(max_new_tokens):
            # Get the predictions and loss.
            logits, loss = self(idx)
            # Focus only on the last time step.
            logits = logits[:, -1, :]  # becomes (B, C)
            # Apply softmax to get probabilities.
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Sample from the distribution.
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append the sampled index to the running sequence.
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

# Create an instance of the BigramLanguageModel with a given vocab size (vocab_size).
m = BigramLanguageModel(vocab_size)

# Calculate logits and loss using input data (xb) and target data (yb).
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# Generate a sequence of tokens based on the initial context and a maximum number of new tokens.
generated_sequence = m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)
print(decode(generated_sequence[0].tolist()))


torch.Size([32, 65])
tensor(4.8170, grad_fn=<NllLossBackward0>)

o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


In [18]:
#Using Adam Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [23]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.563528299331665


In [24]:
#An example of trained bigram model
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



K:
KI bene t wind m ch bapingreng anounn cue e me w w
ROUs nge:

IARDore thecerese,
NENINII:
C$&evea bood ha y mordo owedK:
RI


I at hterenead, wld herrtss g,
Doaswetelan t RGABy ha ban's ser th XEnau len s frut t biveprorikenca s wat ghiQeardow bant GRORDre ts ge l akeingh berire shat buOMENGSThitou micang the g thoe wiestouknce ou d tlandithe CENCRI at
HAne r
Spontit s ean thtint oplls anthes d m, soughenor:

KERE.
burey RI il!
Toulded:
habthe Theere f cre?

BRORDWASS:


Ma ghay fo he fisath


In [25]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [26]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) #xbow: Bag of Words
# Average all elements upto ith row in ith row
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [27]:
wei = torch.tril(torch.ones(T, T)) # Weighted Sums
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C) @ refers to matmul
torch.allclose(xbow, xbow2)

True

In [28]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # Convert 0 to -inf
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [34]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# Single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)  #KEY function
query = nn.Linear(C, head_size, bias=False) #QUERY function
value = nn.Linear(C, head_size, bias=False) #VALUE function
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v #Matmul value of x with wei

out.shape

torch.Size([4, 8, 16])

In [35]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)

#Dividing by sqrt(head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5